# Documentation of Topologies, Bases and Integrals in nutils

This notebook is supposed to help understand the way nutils assembles the (non)linear system of equations affiliated with the numerical solution of a BVP using FE/IGA. It will mainly focus on the way bases and integrals are implemented, thus also requiring further insight into how topologies work in nutils.
This documentation does not postulate to be complete in any sense. It shall just help to gather information and insights on "under the hood"-nutils-mechanisms. Furthermore, the notebook might help to jump into the mechanisms more quickly after a longer period of nutils abstinence ;)
At the beginning, the entire documentation will be restricted to 1D FE.

So let's start with one of nutils' main pillars: topologies (& geometries)

## Topologies and Geometries

Typically, our programs start with the creation of the domain our PDE is defined on. In nutils, a (**topology**, **geometry**) tuple represents this domain. Let us create a simple 1D domain that is spanning from x=0 to x=1 and contains 10 elements as an example: 

In [1]:
from nutils import mesh
import numpy                   # makes linspace available

x0 = 0
x1 = 1
elems = 10
                                                       # 11 nodes... 
domain, geom = mesh.rectilinear([numpy.linspace(x0, x1, elems+1)])

Let us check what kind of objects domain and geom are next.

In [2]:
domain

In [3]:
geom

Array<1>

So our command has produced one StructuredTopology (we'll explain what that is later) and a one-dimensional array (as we only consider one spatial dimension).
In order to fully understand what our object `domain` really is, we will dive into its construction in the next section (headed by a sub-heading). You may, however, skip that part and just remember that domain manages the connectivity of the elements comprising our computational domain.

***
### StructuredTopology - Construction and Characteristics

> Looking into the `mesh.rectilinear` method, we find that the topology is actually built by the following constructor call:
>> `topo = topology.StructuredTopology(root, axes)`

> `root` is the object that eventually leads to the reference mapping for the assembly process. Up to now, it is only an object storing the dimensionality of our domain (1 in this case) as well as the type of reference element ("rect" here). `axes` is simply a tuple of `axis` objects. For every dimension, `axes` contains a separate `axis` object, which is basically a tuple of two ints and one bool. The two ints mark the indices of the first (0) and the last node of the respective domain. In addition, the bool simply indicates whether the domain is periodic or not). So in our case, `root` and `axes` basically look like this:
>> `root = (1, 'rect')          axes = ( (0, 10, False), )`.

> First of all, it is important to know that the *class* `StructuredTopology` is derived from a *parent class* `Topology`. A call to the `StructuredTopology` constructor will, as usual, end with a call to the parent's constructor. In our case, the constructor assigns `root` to the topologies attribute `self.root`, `axes` to `self.axes` and furthermore creates a tuple `self.shape` which simply collects the number of intervals (remember, elements in 1D) for every dimension. Furthermore, the constructor creates a tuple, `self._bnames`, containing the default names of the boundaries of the domain, as we did not specify any names. In the case of our 1D domain, we simply have the two boundaries `'left'` and `'right'`. Lastly, a call to the `Topology` constructor also sets the `self.ndims` attribute to the number of spatial dimensions (1 in our case). **So eventually, our topology object has the following attributes:**
> 1. `self.root = (1, 'rect')               # 1D reference mapping from 'rect' parent domain`
2. `self.axes = ( (0, 10, False), )       # First node has index 0, last one has index 10, nonperiodic domain`
3. `self.nrefine = None                   # Do not refine.`
4. `self.shape = (10, )                   # 1D domain consisting of 10 elements`
5. `self._bnames = ( ('left', 'right'), ) # (x=0) boundary is called 'left', (x=1) boundary is called 'right'`
6. `self.ndims = 1                        # 1D problem`

> **Keep in mind that the connectivity is already saved implicitly, as our topology is of class `StructuredTopology`.**
***

The object `geom` is more complicated. I recommend reading the next section, which dives into its construction process, in order to fully understand this object. If you do not have time for that, try to think of `geom` as a vector valued function mapping from reference coordinates to spatial coordinates. As our example is in 1D, our vector consists of one entry only. Beware of abundant constructor descriptions in the following section!

### Geometry Object - Construction of an ApplyTransforms object

> In the `mesh.rectilinear` method, the `geom` object is created from the following expression:
>> `geom = function.rootcoords(ndims) * scale + offset`

> In our case, `ndims = 1`, `scale` is the length of an element of our equidistant mesh, i.e., `scale = 1/10`, and `offset` is the coordinate of the first node, viz., `offset = 0`. So, we can see that the expression above is nothing but an affine transformation, a (linear) scaling operation (by `scale`) followed by a translation (by `offset`) applied to the return value of `function.rootcoords(1)`. This function call basically reduces to the following constructor call:
>> `return ApplyTransforms(PopHead(1))`.

> The class `ApplyTransforms` lives in the module `function` and is derived from the base class `Array` (this is the type that got displayed after referencing the `geom` object above). Let us now have a look at the definition of the called constructor:
> #### ApplyTransforms constructor
>> `def __init__(self, trans:types.strict[TransformChain], points:strictevaluable=POINTS):
       self.trans = trans
       super().__init__(args=[points, trans], shape=[trans.todims], dtype=float)`
       
> The constructor requires one explicit argument (we do not count self) and provides a default option for the other one. The required argument is of type `TransformChain` (exact characteristics will be covered later) or must be at least an object of a derived class. We will deal with the `POINTS` default argument later and concentrate on this `TransformChain` at first.

> Our call to the constructor passes an object created by the `PopHead` class constructor as our `trans` argument. It is worth noting here that `PopHead` is derived from `TransformChain` (which is in turn derived from `Evaluable`, a central base class in nutils). In order to make things clearer, we will have a look at this constructor as well:
> #### PopHead constructor
>> `def __init__(self, todims:types.strictint, trans=TRANS):
    self.trans = trans
    super().__init__(args=[self.trans], todims=todims)`
    
> We again have one required and one default argument here. The default argument, whose default value is also used in our example as we pass only one argument (`ndims = 1`) into the constructor, is an object living in the namespace of the module `function`. This object, `TRANS`, is a `SelectChain` object (this class is also derived from `TransformChain`) created by passing `0` into the respective constructor. The meaning of the value `0` shall remain unclear until a later point. It is saved in the attribute `self.n` for later use, as we can see in the corresponding constructor:
> #### SelectChain constructor
>> `def __init__(self, n:types.strictint, todims:types.strictint=None):
    self.n = n
    super().__init__(args=[EVALARGS], todims=todims)`
    
> The main thing happening here is that the base class constructor is called passing the arguments `args = [EVALARGS]` and `todims = None`. Well, what exactly is `[EVALARGS]` now? A search quickly reveals its secret identity: It is an object of type `Evaluable` (remember, the base class of `TransformChain` and thus the parent of all the classes considered in this subsection) living in the namespace of the `function` module. It is simply created by the expression `EVALARGS = Evaluable(args=())`. Hence, `EVALARGS` is an `Evaluable` object whose `self._args` attribute is simply set to an empty tuple. Remember, we are currently calling the constructor of `SelectChain`'s base class `TransformChain` in the following manner `TransformChain(args=EVALARGS, todim=None)`. But taking into account that the only attribute `EVALARGS` has is `self._args=()`, we basically pass an empty tuple for `EVALARGS`. Just hold on a little more, we are almost done! First, let us have a look at the respective constructor:
> #### TransformChain constructor
>> `def __init__(self, args:types.tuple[strictevaluable], todims:types.strictint=None):
    self.todims = todims
    super().__init__(args)`

> So we simply set `self.todims=None` and pass our empty-tuple-`EVALARGS` into the constructor of the `function` module grandfather class `Evaluable`, which we will have a look at now:
> #### Evaluable constructor
>> `def __init__(self, args:types.tuple[strictevaluable]):
    super().__init__()
    self.__args = args`

> This last constructor call closes our long chain of constructor calls. Attentive readers might have realized that we are actually using a kind of copy-constructor here (the `EVALARGS` object is actually of type `Evaluable`). Remember that all we did up to now was retrace the origin of the `trans` argument of the `ApplyTransforms` constructor. Rolling the story up the other way round might help to clarify the situation:

> 1. `EVALARGS` is created by the constructor call `EVALARGS = Evaluable(args=())`. Hence, `EVALARGS` only has one attribute, i.e., `EVALARGS._args = ()`. This object is living in the module `function`.
2. In a next step, another `function` module object is created, namely `TRANS`. This is achieved through the call `TRANS = SelectChain(0)`. Now the interesting aspect of the `SelectChain` constructor is that the object `EVALARGS` is always passed to the constructor of the base class `TransformChain`. Hence, `TRANS` has the following attributes: <br>
`TRANS.n = 0
TRANS.todims = None
TRANS._args = ()`
3. A temporary `PopHead` object (we will call it temp for this section) is created (only to be passed into another constructor later on) through the call `PopHead(1, trans=TRANS)`, such that we end up with an object of the following kind: <br>
`temp.trans = TRANS
temp.todims = 1
temp._args  = TRANS   # unsure about this line; may be () instead of TRANS; depends on decorator effect`
4. Implicitly, this `temp` object is passed into the `ApplyTransforms` constructor (whose result we will refer to as app). Besides setting `app.trans = temp`, this also leads to the following call to the `Array` base class constructor `Array(args=[POINTS, temp], shape=[1], dtype = 'float')`.

> Thus, there is one more argument left to be analyzed: `POINTS`. `POINTS` is an object of type `Points` (derived from `Evaluable`, just like `TransformChains`) living in the module `function` that is also created using the `EVALARGS` object. In terms of its attributes, it is like the `TRANS` object described above, but lacking the attributes `TRANS.n` and `TRANS.todims`. Let us have a look at the `Array` constructor in order to entirely demystify the `geom` object:
> #### Array constructor
>> `def __init__(self, args:types.tuple[strictevaluable], shape:asshape, dtype:asdtype):
    self.shape = shape
    self.ndim = len(shape)
    self.dtype = dtype
    super().__init__(args=args)   # Call to Evaluable base class constructor`
    
> Hence, the object `app` finally returned by the call to the `ApplyTransforms` constructor looks like this:
> 1. `app.trans = temp`
2. `app.shape = 1`
3. `app.ndims = 1`
4. `app.dtype = 'float'`
5. `app._args = [POINTS, temp]  # same uncertainty concerning _args attribute as above`

> Inserting the structures of `temp` and `POINTS` into the listing above can give a good idea about what kind of object `app` is.